In [1]:
import sys
sys.path.append("../")


import torch as th
from torchtext import *
import pandas as pd
import sqlite3 as sq

import sys
import os


from loguru import logger

logger.remove()
LOG_FILE_PATH = "../logs/logs.log"
logger.add(sink=LOG_FILE_PATH,format="{time:HH:mm:ss} {message}")
logger.add(sys.stderr,level="INFO")

2

In [2]:
DEVICE = th.device('cuda') if th.cuda.is_available() else th.device('cpu')
logger.info(f"Starting with {DEVICE} device")

2022-03-23 08:50:32.675 | INFO     | __main__:<module>:2 - Starting with cuda device


## Data

In [3]:
from torchtext import data as dta
from torch.utils.data.sampler import SubsetRandomSampler,WeightedRandomSampler
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence
from torch.utils.data import DataLoader,Dataset

datapath = "../data/raw/data.sqlite"

In [4]:
data_con = sq.connect(datapath)
data = pd.read_sql("SELECT * FROM Reviews",data_con)

data

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew read sendak book watch realli rosi movi i...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,fun way children learn month year learn poem t...
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,great littl book read nice rhythm well good re...
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,book poetri month year goe month cute littl po...
...,...,...,...,...,...,...,...,...,...,...,...,...
364166,178145,193174,B009RSR8HO,A4P6AN2L435PV,romarc,0,0,positive,1350432000,LOVE!! LOVE!!,"LOVE, LOVE this sweetener!! I use it in all m...",love love sweeten use bake unsweeten flavor co...
364167,173675,188389,B009SF0TN6,A1L0GWGRK4BYPT,Bety Robinson,0,0,positive,1350518400,Amazing!! Great sauce for everything!,You have to try this sauce to believe it! It s...,tri sauc believ start littl sweet honey tast b...
364168,204727,221795,B009SR4OQ2,A32A6X5KCP7ARG,sicamar,1,1,positive,1350604800,Awesome Taste,I bought this Hazelnut Paste (Nocciola Spread)...,bought hazelnut past nocciola spread local sho...
364169,5259,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...,purchas product local store kid love quick eas...


In [5]:
data.Score.value_counts()/data.shape[0]

positive    0.843178
negative    0.156822
Name: Score, dtype: float64

### Balancing the data

In [6]:
data[data.Score == 'negative'].Score.value_counts()

data = data.append(data[data.Score == 'negative'])
data = data.append(data[data.Score == 'negative'])

In [7]:
data.Score.value_counts()/data.shape[0]

positive    0.573409
negative    0.426591
Name: Score, dtype: float64

In [43]:
data.to_sql('Balanced_Reviews',data_con)

In [4]:
data_con = sq.connect(datapath)
data = pd.read_sql("SELECT * FROM Balanced_Reviews",data_con)

data.Score.value_counts()/data.shape[0]

positive    0.573409
negative    0.426591
Name: Score, dtype: float64

In [5]:
# Ref:- https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets
class TextData(Dataset):
    def __init__(self,datapath):
        logger.debug('Inside TextData')
        
        self.sqldata = self.load_sql_data(datapath) 
        
        self.vocab   = vocab.build_vocab_from_iterator(self.yield_text(self.sqldata),specials=['<unk>'],)
        self.vocab.set_default_index(0)
        self.label_map = {
            "positive":1,
            "negative":0
        }
    
    def yield_text(self,data):
        assert isinstance(data,pd.DataFrame), "data should be object of pandas data frame"
        
        for sentence in data.CleanedText.values:
            yield sentence.split()
            
    def load_sql_data(self,datapath):
        """
            Loads data from sql
            
            Prams:
                datapath: path to sqlite file
        """
        data = None
        
        con = sq.connect(datapath)
        data = pd.read_sql_query("SELECT * FROM Balanced_Reviews",con)
        
        return data
    
    def proper_df(self,data):
        return isinstance(data,pd.DataFrame)
    
    def __len__(self):
        if self.proper_df(self.sqldata):
            logger.info('proper data')
            return len(self.sqldata)
        else:
            logger.info('imporper proper data')

            return len([])
    
    def _map_label(self,label):
        return self.label_map.get(label)
    def __getitem__(self,idx):
        logger.debug('Inside get item')
        
        ## Loading raw text from the dataframe
        raw_text  = self.sqldata.CleanedText.iloc[idx]
        raw_text  = ' '.join(raw_text.split()[:500])
        raw_label = self.sqldata.Score[idx]
        
        ## maping the label to it respective index
        label = self._map_label(raw_label)
        embded_x = self.vocab.lookup_indices(raw_text.split())
        
        logger.debug(f'len of raw_text : {len(raw_text)}')
        embded_x = th.tensor(embded_x)
        
        
        ## Label should be 0 or 1
        assert label in self.label_map.values(), f"Label should in {self.label_map.values()}"
        
        logger.debug(f'returning label {label}')
        return embded_x,label
    

In [9]:
dataloader = TextData(datapath)


In [51]:
# from collections import Counter
# import numpy as np

# label_counter = Counter()

# label_counter.update(dataloader.sqldata.Score.apply(lambda x:1 if x == "positive" else 0).values)

In [52]:
# label_weight = 1/np.array(list(dataloader.sqldata.Score.value_counts().to_dict().values()))
# label_weights= dataloader.sqldata.Score.apply(lambda x:label_weight[0] if x=="positive" else label_weight[1]).values

# label_weights

In [19]:
validation_ratio = 0.2
test_ratio = 0.2
BATCH_SIZE = 25

In [7]:
positive_len = len(dataloader.sqldata[dataloader.sqldata.Score == 'positive'])
negative_len = len(dataloader.sqldata[dataloader.sqldata.Score == 'negative'])
other_len    = len(dataloader.sqldata[~dataloader.sqldata.Score.isin(['negative','positive'])])
total_len    = dataloader.sqldata.shape[0]



logger.info(f'Total number of data points : {total_len}')
logger.info(f'Total number of positive data points : {positive_len} - {round(positive_len/total_len,2) * 100} %')
logger.info(f'Total number of negative data points : {negative_len} - {round(negative_len/total_len,2) * 100} %')
logger.info(f'Total number of other data points : {other_len} - {round(other_len/total_len,2) * 100} %')

2022-03-22 08:17:23.218 | INFO     | __main__:<module>:8 - Total number of data points : 535501
2022-03-22 08:17:23.220 | INFO     | __main__:<module>:9 - Total number of positive data points : 307061 - 56.99999999999999 %
2022-03-22 08:17:23.221 | INFO     | __main__:<module>:10 - Total number of negative data points : 228440 - 43.0 %
2022-03-22 08:17:23.222 | INFO     | __main__:<module>:11 - Total number of other data points : 0 - 0.0 %


In [8]:
test_range = int(total_len * test_ratio)
validation_range = test_range + int(total_len * validation_ratio)

total_inds = [i for i in range(total_len)]

test_inds = total_inds[:test_range]
validate_inds = total_inds[test_range:validation_range]
train_inds = total_inds[validation_range:]

test_range,validation_range

logger.info(f'Total number of train data points {len(train_inds)}')
logger.info(f'Total number of validation data points {len(validate_inds)}')
logger.info(f'Total number of test data points {len(test_inds)}')

2022-03-22 08:17:23.270 | INFO     | __main__:<module>:12 - Total number of train data points 321301
2022-03-22 08:17:23.274 | INFO     | __main__:<module>:13 - Total number of validation data points 107100
2022-03-22 08:17:23.275 | INFO     | __main__:<module>:14 - Total number of test data points 107100


In [26]:
# label_weights

In [57]:

# test_weights = label_weights[:test_range]
# validate_weights = label_weights[test_range:validation_range]
# train_weights = label_weights[validation_range:]

In [9]:
train_sampler = SubsetRandomSampler(train_inds)
validation_sampler = SubsetRandomSampler(validate_inds)
test_sampler = SubsetRandomSampler(test_inds)


In [10]:

def collate_batch(batch):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        processed_text = _text #torch.tensor(text_transform(_text))
        text_list.append(processed_text)
#         logger.debug(f'The label list : {label_list}')
    return (pad_sequence(text_list, padding_value=0.0,batch_first=True),th.tensor(label_list))
#     return (pad_packed_sequence(text_list,batch_first=True,),th.tensor(label_list))


In [11]:
voc_dict = dataloader.vocab.get_stoi()

In [12]:
train_data = DataLoader(dataloader,batch_size=BATCH_SIZE,sampler=train_sampler,collate_fn=collate_batch)
validation_data = DataLoader(dataloader,batch_size=BATCH_SIZE,sampler=validation_sampler,collate_fn=collate_batch)
test_data = DataLoader(dataloader,batch_size=BATCH_SIZE,sampler=test_sampler,collate_fn=collate_batch)

In [62]:
# for i,j in train_data:
#     print(j)
#     break

# weights = [0.9,1]
# num_samples = 50
# ws = WeightedRandomSampler(weights, num_samples, replacement=True, generator=dataloader)
# list(ws)


## Model

In [17]:
from torch import nn

vocab_size = len(dataloader.vocab)

In [14]:


class SeqModel(nn.Module):
        def __init__(self,vocab_size):
                super(SeqModel,self).__init__()

                ## For Embedding we need to pass vocabulary size as first parameter
                ## If the vector representaion for any word is having the number greater than the vocab size
                ## I'll throw index of of range error
                ###### Ex: vocabulary size for our corpus is 500
                ## ##      then the each word representation will any any number that is less than 500 like
                ####       ['this','is','example'] -> [10,45,20]

                self.embd_layer = nn.Embedding(vocab_size,50)
                self.lstm = nn.LSTM(
                                        input_size=50,
                                        hidden_size=100,
                                        num_layers=2,
                                        batch_first=True,
                                        # dropout=0.5
                                )
                self.fc = nn.Linear(100,1)
                self.dropout = nn.Dropout(0.2)
                self.relu = nn.ReLU()
                self.leky = nn.LeakyReLU()
                self.fl = nn.Sigmoid()
        def forward(self,x):
                logger.debug(f'x size before embd : {x.shape}')
                x = self.embd_layer(x)
                logger.debug(f'x size after embd : {x.shape}')
                x = self.dropout(x)
                x_out, (x_hid,x_cell) = self.lstm(x)

                logger.debug(f'x size after lstm : {x.shape}')

                x = self.fc(x_hid[-1])

                logger.debug(f'x size after fc : {x.shape}')
                logger.debug(f'BATCH_SIZE : {BATCH_SIZE}')
                
                logger.debug(f'x size after view : {x.shape}')
                #x = self.relu(x)
                x = self.fl(x)
                logger.debug(f'x size after fl : {x.shape}')
                return x
        
        def checkpoint(self,optimizer,epoch,savepath):

            # ref : https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
            state = {
                        'epoch':epoch,
                        'state_dict':self.state_dict(),
                        'optimizer':optimizer.state_dict()
                    }

            if len(os.listdir(savepath)) < 1:
                torch.save(state,savepath+"checkpoint00")
            else:
                ckpts = []
                for i in os.listdir(savepath):
                    ckpts.append(int(i.split("point")[1]))
                max_ = max(ckpts)
                torch.save(state,savepath+f"checkpoint{max_+1}")


In [30]:


class Optimizer:
    def __init__(self,model):
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters(),lr=0.001)

    def step(self):
        self.optimizer.step()
        

In [23]:
import torch.optim as optim
from tqdm import trange


model = SeqModel(vocab_size)
model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(),)
# optimizer = optim.Adadelta(model.parameters(),)
# optimizer = optim.Adagrad(model.parameters())

In [45]:
from torch.utils.tensorboard import SummaryWriter

tensor_logs = '../tensor_logs/test'
writer = SummaryWriter(tensor_logs)
writer.add_graph(model)

TypeError: 'NoneType' object is not iterable

In [41]:
!tensorboard --logdir=tensor_logs

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [ ]:
EPOCHS = 2
progress = trange(EPOCHS,desc='Epochs',leave=True)

for epoch in progress:
#     optimizer = Optimizer(model)
    for i,(x,y) in enumerate(train_data):
        logger.debug(f'x is {x}')
        logger.debug(f'y is {y}')
        logger.debug(f'x size before embd : {x.shape}')
        logger.debug(f'y size before embd : {y.shape}')
        logger.debug(f'x is on device : {x.device}')
        logger.debug(f'y is on device : {y.device}')
        logger.debug(f'DEVICE is : {DEVICE}')
        x = x.to(DEVICE)#.long()
        y = y.float().to(DEVICE)#.long()


        logger.debug(f'x is on device : {x.device}')
        logger.debug(f'y is on device : {y.device}')


        y_pred = model(x)
        y_pred = y_pred.squeeze()


        # logger.debug(f'y_pred is on device : {y_pred.device}')
        # logger.debug(f'y_pred size : {y_pred.shape}')
        logger.debug(f'y size : {y.shape}')

        loss =  th.nn.BCELoss()(y_pred,y)
        loss.backward()
        optimizer.step()
        if i%BATCH_SIZE == 0:
            progress.set_description(f"Epoch {epoch}")
            progress.set_postfix(batch=f"{int(i/BATCH_SIZE)}/{int(len(train_data)/BATCH_SIZE)}",loss=loss.detach().cpu().numpy())
        
#         writer.add_scalar('train/loss',loss)
    

Epoch 0:   0%|                                                                                                                                                  | 0/2 [22:28<?, ?it/s, batch=139/514, loss=0.650147]

In [94]:
nn.BCELoss()(y_pred,y)

RuntimeError: all elements of input should be between 0 and 1

In [82]:
y_pred.shape,y.float(),y_pred.squeeze()

(torch.Size([25, 1]),
 tensor([1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 0.]),
 tensor([0.0495, 0.0612, 0.0548, 0.0506, 0.0507, 0.0522, 0.0502, 0.0480, 0.0176,
         0.0488, 0.0465, 0.0529, 0.0287, 0.0510, 0.0540, 0.0497, 0.0546, 0.0575,
         0.0466, 0.0487, 0.0537, 0.0473, 0.0402, 0.0518, 0.0475],
        grad_fn=<SqueezeBackward0>))

In [84]:
th.nn.BCELoss()(y_pred.squeeze(),y.float())

tensor(2.7041, grad_fn=<BinaryCrossEntropyBackward0>)

In [40]:
len(train_data)/BATCH_SIZE

loss.detach().cpu().numpy()

array(0.6879633, dtype=float32)

In [326]:
y.shape,outputs.shape

(torch.Size([25]), torch.Size([25, 316]))

In [227]:
text = [1,5,6,9,10,4]
text_tensor = th.tensor(text)
text_embd = nn.Embedding(11,550)(text_tensor)
text_embd,text_embd.shape


nn.Embedding()

(tensor([[ 0.5471,  0.1565,  1.1979,  ...,  1.0553, -0.0148,  0.3221],
         [-0.2483,  0.8593,  0.6033,  ...,  0.1208,  0.1374,  1.2437],
         [-0.2367,  0.9708, -1.6082,  ..., -0.7488,  0.7943, -0.9277],
         [-0.2510, -0.0627, -0.1987,  ...,  0.5655,  0.3042,  0.4546],
         [-0.0975,  0.4808,  0.6477,  ...,  0.8855, -0.3269,  0.3226],
         [ 1.8746, -0.1865, -0.2328,  ...,  1.3510, -1.0670,  0.7845]],
        grad_fn=<EmbeddingBackward0>),
 torch.Size([6, 550]))

In [1]:
dataloader.vocab.lookup_indices([0])

NameError: name 'dataloader' is not defined

In [28]:
dataloader.vocab.lookup_indices(['test'])
dataloader.vocab.lookup_token(5)

## Used to get list of tokens given a list of indixes
dataloader.vocab.lookup_tokens([1,2,5])

['like', 'tast', 'product']

In [55]:
dummy_text = dataloader.sqldata.CleanedText.loc[0]
dummy_Vect = dataloader.vocab.lookup_indices(dummy_text.split())
th.tensor(dummy_Vect).shape
th.tensor(dummy_Vect).broadcast_to((1,1,-1)).shape
dataloader.vocab.get_itos()

# th.tensor(dummy_Vect).__dir__()

torch.Size([1, 1, 35])

In [62]:
sorted(dataloader.vocab.get_stoi().items(), key = lambda x:x[1])

[('like', 0),
 ('tast', 1),
 ('flavor', 2),
 ('good', 3),
 ('product', 4),
 ('use', 5),
 ('one', 6),
 ('love', 7),
 ('great', 8),
 ('tri', 9),
 ('tea', 10),
 ('coffe', 11),
 ('get', 12),
 ('make', 13),
 ('food', 14),
 ('would', 15),
 ('buy', 16),
 ('time', 17),
 ('realli', 18),
 ('eat', 19),
 ('amazon', 20),
 ('order', 21),
 ('dont', 22),
 ('much', 23),
 ('price', 24),
 ('also', 25),
 ('find', 26),
 ('littl', 27),
 ('bag', 28),
 ('best', 29),
 ('dog', 30),
 ('even', 31),
 ('drink', 32),
 ('well', 33),
 ('store', 34),
 ('ive', 35),
 ('better', 36),
 ('chocol', 37),
 ('box', 38),
 ('mix', 39),
 ('day', 40),
 ('water', 41),
 ('sugar', 42),
 ('year', 43),
 ('recommend', 44),
 ('look', 45),
 ('sweet', 46),
 ('first', 47),
 ('want', 48),
 ('packag', 49),
 ('cup', 50),
 ('give', 51),
 ('brand', 52),
 ('found', 53),
 ('purchas', 54),
 ('high', 55),
 ('think', 56),
 ('treat', 57),
 ('made', 58),
 ('work', 59),
 ('way', 60),
 ('bought', 61),
 ('enjoy', 62),
 ('say', 63),
 ('need', 64),
 ('know',

In [11]:
for i,z in data:
#     print(i)
#     break
    pass

2022-03-07 09:24:24.885 | INFO     | __main__:__len__:38 - proper data
2022-03-07 09:24:24.886 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:24.887 | INFO     | __main__:__getitem__:58 - embded_x : [15004, 28, 1082, 14, 369, 2406, 3183, 13772, 1117, 1214, 543, 103, 3784, 7672, 846, 793, 10437, 1817, 11855, 8, 171, 781, 1082, 1054, 2797, 1213, 1082, 1482, 1862, 369, 82, 258, 13772, 1407, 1661]
2022-03-07 09:24:24.887 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:24.888 | INFO     | __main__:__getitem__:58 - embded_x : [1035, 232, 12651, 1082, 661, 19, 8057, 1216, 2702, 8, 369, 8, 141, 594, 128, 640, 378, 12261, 106, 212, 3272, 91, 71, 267, 84, 1639, 149]
2022-03-07 09:24:24.888 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:24.889 | INFO     | __main__:__getitem__:58 - embded_x : [694, 61, 826, 793, 130, 44, 793, 9545, 1414, 1002, 44, 1, 51196, 2955, 9545]
2022-03-07 09:24:24.889 | INFO     | __main__:__getitem

2022-03-07 09:24:24.959 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:24.960 | INFO     | __main__:__getitem__:58 - embded_x : [7, 9358, 1407, 1082, 818, 232, 1286, 232, 7, 3290, 287, 762, 712, 217, 16, 1735, 178, 1082, 73, 6, 1082, 99, 1671, 3607, 2049, 90, 2165, 976, 8, 453, 232, 131, 633, 2951, 5073, 3784, 4711, 130, 44, 32, 12917, 12651, 349, 63, 7, 360, 14823]
2022-03-07 09:24:24.961 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:24.962 | INFO     | __main__:__getitem__:58 - embded_x : [2584, 415, 79, 1082, 232, 8, 9031, 1419, 1658, 139, 63, 232]
2022-03-07 09:24:24.963 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:24.967 | INFO     | __main__:__getitem__:58 - embded_x : [3376, 477, 208, 782, 116, 359, 1435, 312, 4181, 1082, 12917, 12651, 113, 28, 1082, 130, 8922, 251, 2797, 24, 694, 232, 540, 1918, 976, 49, 14, 131, 28, 7, 1082, 836, 1419, 61, 3607, 130, 44, 596, 3245, 341, 8, 217, 695, 3871, 61, 12651]


2022-03-07 09:24:25.040 | INFO     | __main__:__getitem__:58 - embded_x : [31, 8, 58, 150, 6, 868, 1196, 58, 125]
2022-03-07 09:24:25.043 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.044 | INFO     | __main__:__getitem__:58 - embded_x : [9, 58, 280, 31, 251, 8, 22, 133, 25, 9]
2022-03-07 09:24:25.045 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.046 | INFO     | __main__:__getitem__:58 - embded_x : [17, 13036, 590, 199, 336, 54, 397, 31, 8, 172]
2022-03-07 09:24:25.047 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.051 | INFO     | __main__:__getitem__:58 - embded_x : [125, 4, 87, 31, 9, 868, 21331, 91, 2219, 305, 65, 4, 394]
2022-03-07 09:24:25.052 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.053 | INFO     | __main__:__getitem__:58 - embded_x : [71, 31, 8, 5, 8, 287, 1084, 634, 701, 1173, 287, 246, 5113, 2044, 277, 386, 597, 246, 23, 49, 52, 24, 6496, 13, 7, 58, 14, 24

2022-03-07 09:24:25.111 | INFO     | __main__:__getitem__:58 - embded_x : [497, 76, 199, 56, 112, 510, 424, 4, 868, 58, 56, 142, 56, 45]
2022-03-07 09:24:25.111 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.112 | INFO     | __main__:__getitem__:58 - embded_x : [71, 1441, 2135, 6, 868, 58, 26, 6, 1572, 23, 63, 24, 66, 13, 58, 10, 99, 58, 234, 30]
2022-03-07 09:24:25.113 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.114 | INFO     | __main__:__getitem__:58 - embded_x : [531, 165, 2729, 400, 58, 868, 9, 394, 246, 269, 386, 477, 246, 133, 868, 1196]
2022-03-07 09:24:25.116 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.116 | INFO     | __main__:__getitem__:58 - embded_x : [1413, 883, 1, 1288, 31, 14783, 1282, 868, 31, 2925, 194, 344, 1413, 808, 386, 883, 707, 43051, 6, 868, 1645, 64, 8, 4, 31]
2022-03-07 09:24:25.117 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.117 | INFO    

2022-03-07 09:24:25.172 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.175 | INFO     | __main__:__getitem__:58 - embded_x : [6, 17, 2703, 334, 358, 266, 46, 1188, 336, 54, 21, 31, 8]
2022-03-07 09:24:25.176 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.177 | INFO     | __main__:__getitem__:58 - embded_x : [31, 2933, 857, 679, 526, 1413, 58, 14, 52, 857, 649, 119, 1866, 1322, 31]
2022-03-07 09:24:25.178 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:25.179 | INFO     | __main__:__getitem__:58 - embded_x : [5, 179, 1366, 474, 287, 31, 8, 1413, 36, 6, 354, 169, 31, 58, 5, 6, 31, 31, 431, 264, 66, 846, 1645, 460, 1413, 58, 775, 1804, 52, 246, 190, 41, 6, 520, 122, 124, 4, 5, 98, 246, 923, 140, 570, 1115, 140, 58, 129, 4599, 246, 70, 570, 244, 117, 561, 477, 246, 1583, 7401, 3405, 1486, 130, 145, 8, 10, 5, 2178, 145, 6336, 2466, 168, 1462, 527, 425, 251, 207, 425, 1501, 279, 2, 7, 58, 17, 31, 8]
2022-03-07 09

...

2022-03-07 09:24:34.415 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:34.418 | INFO     | __main__:__getitem__:58 - embded_x : [100, 1196, 530, 978, 152, 1265, 671, 6, 3, 15, 36, 664, 44, 145, 2217, 940, 832, 96, 57, 91, 57, 48, 100, 363, 303, 379, 1943, 781, 5599, 2421, 632, 882, 579, 41, 3976]
2022-03-07 09:24:34.419 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:34.420 | INFO     | __main__:__getitem__:58 - embded_x : [2981, 96, 99, 10, 10, 1124, 48, 104, 364, 146, 4101, 303, 86, 194, 2432, 2048, 18591, 380, 2048, 694, 96, 19698, 860, 61, 1289, 826, 174, 982, 160]
2022-03-07 09:24:34.421 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:34.422 | INFO     | __main__:__getitem__:58 - embded_x : [720, 343, 4049, 64, 290, 6, 100, 30, 40, 108, 348, 790, 196, 2156, 26, 6, 2735, 2041, 100, 671, 366, 481, 23318, 133, 305, 816, 2134, 2836, 422, 1080, 2998, 345, 819, 152, 224, 2134, 2, 4, 497, 290, 24, 6]
2022-03-07 09:2

2022-03-07 09:24:34.471 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:34.472 | INFO     | __main__:__getitem__:58 - embded_x : [8, 96, 1007, 147, 96, 162, 739, 3, 86, 633]
2022-03-07 09:24:34.473 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:34.476 | INFO     | __main__:__getitem__:58 - embded_x : [3399, 5275, 983, 1050, 10284, 5010, 4583, 555, 96, 1061, 1257, 22793, 3262, 3052, 363, 104, 821, 386, 28, 420, 102, 36, 54, 30, 2261, 170, 959, 4976, 293, 368, 120, 5296, 419, 28, 1388, 555, 19, 1252, 96, 14, 131, 328, 662, 1412, 29, 99, 368, 2058, 32, 2092, 17383, 96, 262, 6, 2246, 3869, 14, 131, 936, 267, 442, 632, 1252, 955, 194, 348, 814, 458, 352, 10336, 10336, 96, 30, 571, 407, 4035, 3052, 318, 6938, 126, 328, 170, 143, 1, 4035, 1252, 454]
2022-03-07 09:24:34.479 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:34.482 | INFO     | __main__:__getitem__:58 - embded_x : [3262, 338, 64, 611, 1257, 318, 30, 118, 18, 

2022-03-07 09:24:34.536 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:34.537 | INFO     | __main__:__getitem__:58 - embded_x : [275, 2400, 579, 1185, 505, 2400, 132, 70, 974, 990, 174, 57, 460, 1374, 570, 1011, 100, 3584, 2400, 1491, 988, 995, 959, 351, 804, 100, 14, 67, 1982, 1055, 221, 118, 6411, 2400, 100, 6411, 2400, 100, 421, 32061, 2411, 3584, 100, 75, 80, 1236, 108, 46, 628, 1236, 1627, 192, 259, 74, 76, 586, 80, 3, 959, 23, 277, 912, 259, 1456, 606, 1260, 118, 662, 262, 766, 4406, 2203, 4446, 100, 120, 565, 1119, 934, 2750, 3787, 4406, 52, 3559, 1622, 3, 26, 74, 100, 1164, 120, 83, 174, 354, 4406, 419, 325, 1425, 499, 89, 67, 959, 379, 2, 1513, 811, 795, 118, 140, 6411, 2400, 100, 402, 100, 122, 43, 214, 97, 1627, 118, 52, 1687, 735, 312, 97, 804, 144, 370, 259, 1620, 13472, 360, 100, 7150, 5233, 6411, 816, 583, 1845, 228, 1244, 14, 100, 115, 1, 4, 3584, 2400, 4016, 867, 476, 63, 6411, 2400, 100, 622, 204]
2022-03-07 09:24:34.538 | INFO     | __main__:

2022-03-07 09:24:34.596 | INFO     | __main__:__getitem__:58 - embded_x : [54, 89, 100, 465, 145, 1276, 13100, 1947, 35, 273, 100, 99, 212, 453, 2463, 60, 79, 441, 3105, 13625, 22, 210, 105, 760, 100, 236, 884, 3, 67, 26, 55, 3, 8671, 13625, 19, 89, 9302, 11145, 690, 363, 457, 11145, 499, 89, 800, 9, 617, 714, 702, 185, 10, 8, 89, 100, 487, 79]
2022-03-07 09:24:34.597 | INFO     | __main__:__getitem__:48 - Inside get item
2022-03-07 09:24:34.598 | INFO     | __main__:__getitem__:58 - embded_x : [36, 6, 593, 593, 44, 267, 79, 89, 100, 1541, 48, 2031, 924, 593, 159, 4, 307, 22, 978, 4, 89, 100, 52, 10, 4, 1752, 709, 275, 593, 1, 89, 89, 100, 338, 10, 593, 85, 592, 3, 363, 83, 19, 89, 100, 89, 859, 3, 643, 89, 100, 267, 417, 18, 748, 3, 28, 363, 20, 593, 259, 1179, 821, 1667, 198, 759, 65, 28, 739, 391, 138, 591, 1950, 100, 706, 1504, 263, 234, 21, 16, 990, 17, 210, 412, 1159, 978, 4, 52, 218, 118, 224, 268, 52, 174, 149, 6501, 8, 89, 100, 180, 86, 91, 363, 990, 13, 593, 63]
2022-03-07 09

KeyboardInterrupt: 

In [13]:
# dataloader.vocab.forward(text[0].split())


# dataloader.vocab.get_itos(['this'])

i.shape

torch.Size([1, 27])

In [1]:
from torchtext.legacy import data

In [84]:
dataloader.sqldata

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew read sendak book watch realli rosi movi i...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,fun way children learn month year learn poem t...
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,great littl book read nice rhythm well good re...
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,book poetri month year goe month cute littl po...
...,...,...,...,...,...,...,...,...,...,...,...,...
364166,178145,193174,B009RSR8HO,A4P6AN2L435PV,romarc,0,0,positive,1350432000,LOVE!! LOVE!!,"LOVE, LOVE this sweetener!! I use it in all m...",love love sweeten use bake unsweeten flavor co...
364167,173675,188389,B009SF0TN6,A1L0GWGRK4BYPT,Bety Robinson,0,0,positive,1350518400,Amazing!! Great sauce for everything!,You have to try this sauce to believe it! It s...,tri sauc believ start littl sweet honey tast b...
364168,204727,221795,B009SR4OQ2,A32A6X5KCP7ARG,sicamar,1,1,positive,1350604800,Awesome Taste,I bought this Hazelnut Paste (Nocciola Spread)...,bought hazelnut past nocciola spread local sho...
364169,5259,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...,purchas product local store kid love quick eas...


In [86]:
text = ['This is testisng','what is this testing']

def yield_text(text):
    for t in text:
        yield t.split()
        
next(yield_text(dataloader.sqldata.CleanedText.values))

['witti',
 'littl',
 'book',
 'make',
 'son',
 'laugh',
 'loud',
 'recit',
 'car',
 'drive',
 'along',
 'alway',
 'sing',
 'refrain',
 'hes',
 'learn',
 'whale',
 'india',
 'droop',
 'love',
 'new',
 'word',
 'book',
 'introduc',
 'silli',
 'classic',
 'book',
 'will',
 'bet',
 'son',
 'still',
 'abl',
 'recit',
 'memori',
 'colleg']

In [76]:
voc = vocab.build_vocab_from_iterator(yield_text(text), specials=["<unk>"])

In [75]:
voc.set_default_index(0)

In [83]:
voc.lookup_indices(text[0].split())
voc.lookup_indices(['<unk>'])
# voc.lookup_token(0)

[0]

## PyTorch Lightning
`PyTorch`

In [6]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from torch import nn


from loguru import logger

logger.remove()
LOG_FILE_PATH = "../logs/logs.log"
logger.add(sink=LOG_FILE_PATH,format="{time:HH:mm:ss} {message}")
logger.add(sys.stderr,level="INFO")

4

In [7]:
dataloader = TextData(datapath)
vocab_size = len(dataloader.vocab)
BATCH_SIZE = 25

### Data Loader

In [8]:
class LiteDateLoad(pl.LightningDataModule):
    def __init__(self,
                 data_path,
                 batch_size,
                 dataloader,
                 val_ratio = 0.2,
                 test_ratio = 0.2
                ):
        super(LiteDateLoad,self).__init__()
        self.data_path = data_path
        self.batch_size = batch_size
        self.val_ratio = val_ratio
        self.test_ratio = test_ratio
        self.dataloader = dataloader
    
    
    def collate_batch(self,batch):
        label_list, text_list = [], []
        for (_text, _label) in batch:
            label_list.append(_label)
            processed_text = _text #torch.tensor(text_transform(_text))
            text_list.append(processed_text)
    
        return (pad_sequence(text_list, padding_value=0.0,batch_first=True),th.tensor(label_list))

    
    
    def setup(self):
        
        total_len    = self.dataloader.sqldata.shape[0]
        
        test_range = int(total_len * self.test_ratio)
        validation_range = test_range + int(total_len * self.val_ratio)

        total_inds = [i for i in range(total_len)]

        test_inds = total_inds[:test_range]
        validate_inds = total_inds[test_range:validation_range]
        train_inds = total_inds[validation_range:]
        




        self.train_sampler = SubsetRandomSampler(train_inds)
        self.validation_sampler = SubsetRandomSampler(validate_inds)
        self.test_sampler = SubsetRandomSampler(test_inds)
        
        
        
    def train_dataloader(self):
        return  DataLoader(
                            self.dataloader,batch_size=self.batch_size,
                             sampler=self.train_sampler,
                             collate_fn=self.collate_batch
                            )
    
    def val_dataloader(self):
        return  DataLoader(
                            self.dataloader,batch_size=self.batch_size,
                            sampler=self.validation_sampler,
                            collate_fn=self.collate_batch
                            )
    
    def test_dataloader(self):
        return  DataLoader(
                            self.dataloader,batch_size=self.batch_size,
                            sampler=self.test_sampler,
                            collate_fn=self.collate_batch
                        )

In [9]:
class SeqModel(pl.LightningModule):
        def __init__(self,vocab_size,dataloader):
                super(SeqModel,self).__init__()

                ## For Embedding we need to pass vocabulary size as first parameter
                ## If the vector representaion for any word is having the number greater than the vocab size
                ## I'll throw index of of range error
                ###### Ex: vocabulary size for our corpus is 500
                ## ##      then the each word representation will any any number that is less than 500 like
                ####       ['this','is','example'] -> [10,45,20]

                self.embd_layer = nn.Embedding(vocab_size,50)
                self.lstm = nn.LSTM(
                                        input_size=50,
                                        hidden_size=100,
                                        num_layers=2,
                                        batch_first=True,
                                        # dropout=0.5
                                )
                self.fc = nn.Linear(100,1)
                self.dropout = nn.Dropout(0.2)
                self.relu = nn.ReLU()
                self.leky = nn.LeakyReLU()
                self.fl = nn.Sigmoid()
                self.dataloader = dataloader
                self.automatic_optimization = False
                
                
        def model(self,x):
                logger.debug(f'x size before embd : {x.shape}')
                x = self.embd_layer(x)
                logger.debug(f'x size after embd : {x.shape}')
                x = self.dropout(x)
                x_out, (x_hid,x_cell) = self.lstm(x)

                logger.debug(f'x size after lstm : {x.shape}')

                x = self.fc(x_hid[-1])

                logger.debug(f'x size after fc : {x.shape}')
                logger.debug(f'BATCH_SIZE : {BATCH_SIZE}')
                
                logger.debug(f'x size after view : {x.shape}')
                #x = self.relu(x)
                #x = self.fl(x)
                logger.debug(f'x size after fl : {x.shape}')
                return x
        
        def forward(self,x):
            x = self.model(x)
            return x
        
        def training_step(self,batch,idx):
            
            opt = self.optimizers()
            x,y = batch
            x = x.to(DEVICE)#.long()
            y = y.float().to(DEVICE)#.long()


            logger.info(f'x is on device : {x.device}')
            logger.info(f'y is on device : {y.device}')


            y_pred = self(x)
            
            logger.info(f'y_pred size before sqeeze is : {y_pred.shape}')
            logger.info(f'y_pred before sqeeze is : {y_pred}')
            y_pred = y_pred.squeeze()


            logger.info(f'y_pred is : {y_pred}')
            logger.info(f'y_pred size : {y_pred.shape}')
            logger.info(f'y size : {y.shape}')
            logger.info(f'y is : {y}')

            loss =  th.nn.BCELoss()(y_pred,y)
            self.manual_backward(loss)
            opt.step()
            return loss
        
#             loss.backward()
#             optimizer.step()
#             if i%BATCH_SIZE == 0:
#                 progress.set_description(f"Epoch {epoch}")
#                 progress.set_postfix(batch=f"{int(i/BATCH_SIZE)}/{int(len(train_data)/BATCH_SIZE)}",loss=loss.detach().cpu().numpy())

            
        def train_dataloader(self):
            return self.dataloader.train_dataloader()
        
        
        def configure_optimizers(self):
            return th.optim.Adam(self.parameters(), lr=0.02)

        def checkpoint(self,optimizer,epoch,savepath):

            # ref : https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
            state = {
                        'epoch':epoch,
                        'state_dict':self.state_dict(),
                        'optimizer':optimizer.state_dict()
                    }

            if len(os.listdir(savepath)) < 1:
                torch.save(state,savepath+"checkpoint00")
            else:
                ckpts = []
                for i in os.listdir(savepath):
                    ckpts.append(int(i.split("point")[1]))
                max_ = max(ckpts)
                torch.save(state,savepath+f"checkpoint{max_+1}")


In [14]:
AVAIL_GPUS = min(1, th.cuda.device_count())
AVAIL_GPUS

1

In [10]:
dt_loader = LiteDateLoad(datapath,BATCH_SIZE,dataloader)
dt_loader.setup()


In [11]:
# dt_loader.train_dataloader()

In [12]:
# !tensorboard --logdir="./lightning_logs/"

In [43]:
y_pred = th.tensor([[-0.0301, -0.0367, -0.0485, -0.0489, -0.0303, -0.0366, -0.0054, -0.0465,
        -0.0477, -0.0386, -0.0345, -0.0261, -0.0269, -0.0273, -0.0507, -0.0399,
        -0.0463, -0.0380, -0.0335, -0.0391, -0.0179, -0.0378, -0.0175, -0.0442,
        -0.0285]], device='cpu')



y_pred = th.tensor([[-0.0131],
        [-0.0000001],
        [-0.0095],
        [-0.0101],
        [-0.0182],
        [-0.0084],
        [-0.0215],
        [-0.0086],
        [ 0.0045],
        [-0.0273],
        [-0.0042],
        [-0.0129],
        [-0.0119],
        [-0.0232],
        [-0.0101],
        [-0.0175],
        [-0.0231],
        [-0.0072],
        [-0.0208],
        [-0.0120],
        [-0.0086],
        [-0.0255],
        [-0.0256],
        [ 0.0248],
        [9129]], device='cpu')

logger.info(y_pred.shape)
# y_pred = y_pred.squeeze()
y_pred = nn.Sigmoid()(y_pred)

y_pred.squeeze().int()

# y_pred

2022-03-23 09:03:50.042 | INFO     | __main__:<module>:34 - torch.Size([25, 1])


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1], dtype=torch.int32)

In [30]:
th.argmax(y_pred,dim=0)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

In [18]:
m = nn.Sigmoid()
input_ = th.randn(2)
output = m(input_)
output

tensor([0.4240, 0.4453])

In [11]:
model = SeqModel(vocab_size,dt_loader)
model_logger = TensorBoardLogger('./lightning_logs/','first_model')
trainer = pl.Trainer(
    gpus=1,
    max_epochs=2,
    progress_bar_refresh_rate=20,
    logger=model_logger,
    
)
trainer.fit(model)


/home/hemanth/anaconda3/envs/pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | embd_layer | Embedding | 3.6 M 
1 | lstm       | LSTM      | 141 K 
2 | fc         | Linear    | 101   
3 | dropout    | Dropout   | 0     
4 | relu       | ReLU      | 0     
5 | leky       | LeakyReLU | 0     
6 | fl         | Sigmoid   | 0     
-----------

Training: 0it [00:00, ?it/s]

2022-03-23 08:51:05.111 | INFO     | __main__:training_step:61 - x is on device : cuda:0
2022-03-23 08:51:05.112 | INFO     | __main__:training_step:62 - y is on device : cuda:0
2022-03-23 08:51:05.118 | INFO     | __main__:training_step:67 - y_pred size before sqeeze is : torch.Size([25, 1])
2022-03-23 08:51:05.122 | INFO     | __main__:training_step:68 - y_pred before sqeeze is : tensor([[-0.0131],
        [-0.0156],
        [-0.0095],
        [-0.0101],
        [-0.0182],
        [-0.0084],
        [-0.0215],
        [-0.0086],
        [ 0.0045],
        [-0.0273],
        [-0.0042],
        [-0.0129],
        [-0.0119],
        [-0.0232],
        [-0.0101],
        [-0.0175],
        [-0.0231],
        [-0.0072],
        [-0.0208],
        [-0.0120],
        [-0.0086],
        [-0.0255],
        [-0.0256],
        [ 0.0248],
        [-0.0129]], device='cuda:0', grad_fn=<AddmmBackward0>)
2022-03-23 08:51:05.125 | INFO     | __main__:training_step:72 - y_pred is : tensor([-0.0131, -0

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [20]:
th.cuda.empty_cache()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.